# Chapter 26: Advanced Component Patterns

As applications grow in complexity, component design patterns become essential for creating flexible, reusable, and maintainable UI architectures. Advanced patterns like Compound Components, Render Props, and Headless UI allow you to build components that are both powerful and adaptable to varying requirements without sacrificing type safety or performance.

By the end of this chapter, you'll master Compound Components for intuitive APIs, Render Props for dynamic behavior injection, Higher-Order Components for cross-cutting concerns, Custom Hooks for logic reuse, Headless UI for behavior without styling, Server Component composition patterns specific to the App Router, and performance-optimized component architectures.

## 26.1 Compound Components

Compound Components expose a flexible API where parent and child components work together implicitly through shared state.

### Building a Compound Component System

Create a `Select` component with a flexible composition API:

```typescript
// components/ui/select/index.tsx
'use client';

import { 
  createContext, 
  useContext, 
  useState, 
  useCallback,
  ReactNode 
} from 'react';

interface SelectContextType {
  value: string;
  onChange: (value: string) => void;
  isOpen: boolean;
  setIsOpen: (open: boolean) => void;
}

const SelectContext = createContext<SelectContextType | null>(null);

const useSelect = () => {
  const context = useContext(SelectContext);
  if (!context) throw new Error('Select components must be used within Select.Root');
  return context;
};

// Root component
interface RootProps {
  value: string;
  onChange: (value: string) => void;
  children: ReactNode;
}

function Root({ value, onChange, children }: RootProps) {
  const [isOpen, setIsOpen] = useState(false);
  
  const handleChange = useCallback((newValue: string) => {
    onChange(newValue);
    setIsOpen(false);
  }, [onChange]);
  
  return (
    <SelectContext.Provider value={{ value, onChange: handleChange, isOpen, setIsOpen }}>
      <div className="relative">{children}</div>
    </SelectContext.Provider>
  );
}

// Trigger component
function Trigger({ children }: { children: ReactNode }) {
  const { value, isOpen, setIsOpen } = useSelect();
  
  return (
    <button
      type="button"
      onClick={() => setIsOpen(!isOpen)}
      className="w-full px-4 py-2 border rounded bg-white text-left"
      aria-expanded={isOpen}
    >
      {children || value || 'Select an option'}
    </button>
  );
}

// Content/Menu component
function Content({ children }: { children: ReactNode }) {
  const { isOpen } = useSelect();
  
  if (!isOpen) return null;
  
  return (
    <div className="absolute z-50 w-full mt-1 bg-white border rounded-md shadow-lg">
      {children}
    </div>
  );
}

// Item component
interface ItemProps {
  value: string;
  children: ReactNode;
}

function Item({ value, children }: ItemProps) {
  const { value: selectedValue, onChange } = useSelect();
  const isSelected = selectedValue === value;
  
  return (
    <div
      role="option"
      aria-selected={isSelected}
      onClick={() => onChange(value)}
      className={`px-4 py-2 cursor-pointer hover:bg-gray-100 ${
        isSelected ? 'bg-blue-50 text-blue-600' : ''
      }`}
    >
      {children}
    </div>
  );
}

// Export as namespace
export const Select = {
  Root,
  Trigger,
  Content,
  Item,
};
```

### Usage Example

The compound pattern provides an intuitive, declarative API:

```typescript
// app/page.tsx
'use client';

import { Select } from '@/components/ui/select';
import { useState } from 'react';

export default function Page() {
  const [value, setValue] = useState('');
  
  return (
    <Select.Root value={value} onChange={setValue}>
      <Select.Trigger>Choose a fruit</Select.Trigger>
      <Select.Content>
        <Select.Item value="apple">Apple</Select.Item>
        <Select.Item value="banana">Banana</Select.Item>
        <Select.Item value="orange">Orange</Select.Item>
      </Select.Content>
    </Select.Root>
  );
}
```

### Advanced Compound Pattern with Form Integration

Integrate with React Hook Form seamlessly:

```typescript
// components/ui/form-select.tsx
'use client';

import { Select } from './select';
import { useController, Control } from 'react-hook-form';

interface FormSelectProps {
  name: string;
  control: Control<any>;
  label: string;
  options: { value: string; label: string }[];
}

export function FormSelect({ name, control, label, options }: FormSelectProps) {
  const {
    field: { value, onChange },
    fieldState: { error },
  } = useController({ name, control });
  
  return (
    <div className="space-y-1">
      <label className="text-sm font-medium">{label}</label>
      <Select.Root value={value} onChange={onChange}>
        <Select.Trigger />
        <Select.Content>
          {options.map((option) => (
            <Select.Item key={option.value} value={option.value}>
              {option.label}
            </Select.Item>
          ))}
        </Select.Content>
      </Select.Root>
      {error && <p className="text-sm text-red-500">{error.message}</p>}
    </div>
  );
}
```

## 26.2 Render Props

The Render Props pattern allows components to share code and behavior through a function prop.

### Dynamic Layout Components

Create flexible layout components that delegate rendering:

```typescript
// components/data-list.tsx
interface DataListProps<T> {
  data: T[];
  renderItem: (item: T, index: number) => React.ReactNode;
  renderEmpty?: () => React.ReactNode;
  renderLoading?: () => React.ReactNode;
  isLoading?: boolean;
  keyExtractor: (item: T) => string;
}

export function DataList<T>({
  data,
  renderItem,
  renderEmpty,
  renderLoading,
  isLoading,
  keyExtractor,
}: DataListProps<T>) {
  if (isLoading && renderLoading) {
    return <>{renderLoading()}</>;
  }
  
  if (data.length === 0 && renderEmpty) {
    return <>{renderEmpty()}</>;
  }
  
  return (
    <div className="space-y-2">
      {data.map((item, index) => (
        <div key={keyExtractor(item)}>
          {renderItem(item, index)}
        </div>
      ))}
    </div>
  );
}

// Usage with different render strategies
// Strategy 1: Card layout
<DataList
  data={users}
  keyExtractor={(u) => u.id}
  renderItem={(user) => (
    <UserCard name={user.name} email={user.email} />
  )}
  renderEmpty={() => <EmptyState icon="users" message="No users found" />}
/>

// Strategy 2: Table row layout (same component, different rendering)
<DataList
  data={users}
  keyExtractor={(u) => u.id}
  renderItem={(user) => (
    <tr>
      <td>{user.name}</td>
      <td>{user.email}</td>
      <td>{user.role}</td>
    </tr>
  )}
  renderEmpty={() => (
    <tr>
      <td colSpan={3}>No data available</td>
    </tr>
  )}
/>
```

### Mouse Position Tracker

Demonstrate cross-cutting concerns with render props:

```typescript
// components/mouse-tracker.tsx
'use client';

import { useState, useEffect, ReactNode } from 'react';

interface MousePosition {
  x: number;
  y: number;
}

interface MouseTrackerProps {
  render: (position: MousePosition) => ReactNode;
}

export function MouseTracker({ render }: MouseTrackerProps) {
  const [position, setPosition] = useState<MousePosition>({ x: 0, y: 0 });
  
  useEffect(() => {
    const handleMouseMove = (e: MouseEvent) => {
      setPosition({ x: e.clientX, y: e.clientY });
    };
    
    window.addEventListener('mousemove', handleMouseMove);
    return () => window.removeEventListener('mousemove', handleMouseMove);
  }, []);
  
  return <>{render(position)}</>;
}

// Usage
<MouseTracker 
  render={({ x, y }) => (
    <div style={{ position: 'fixed', left: x, top: y }}>
      Tooltip at {x}, {y}
    </div>
  )}
/>
```

### WithRenderProps HOC Helper

Type-safe helper for render props:

```typescript
// lib/render-props.ts
import { ComponentType } from 'react';

export type WithRenderProps<P, R> = P & {
  render: (props: R) => React.ReactNode;
};

export function createRenderPropsComponent<P, R>(
  useLogic: (props: P) => R
) {
  return function RenderPropsComponent({
    render,
    ...props
  }: WithRenderProps<P, R>) {
    const logic = useLogic(props as P);
    return <>{render(logic)}</>;
  };
}

// Usage
// components/with-fetch.tsx
import { createRenderPropsComponent } from '@/lib/render-props';

interface FetchProps {
  url: string;
  options?: RequestInit;
}

interface FetchResult {
  data: any;
  isLoading: boolean;
  error: Error | null;
  refetch: () => void;
}

const useFetch = ({ url, options }: FetchProps): FetchResult => {
  // Implementation...
  return { data, isLoading, error, refetch };
};

export const WithFetch = createRenderPropsComponent<FetchProps, FetchResult>(useFetch);

// Usage
<WithFetch 
  url="/api/users"
  render={({ data, isLoading, error }) => {
    if (isLoading) return <Spinner />;
    if (error) return <ErrorMessage error={error} />;
    return <UserList users={data} />;
  }}
/>
```

## 26.3 Higher-Order Components

HOCs wrap components to provide additional props or behavior, useful for cross-cutting concerns.

### Authentication HOC

Protect components with authentication logic:

```typescript
// components/with-auth.tsx
import { ComponentType } from 'react';
import { useSession } from 'next-auth/react';
import { useRouter } from 'next/navigation';

interface WithAuthProps {
  user: {
    id: string;
    email: string;
    role: string;
  };
}

export function withAuth<P extends WithAuthProps>(
  WrappedComponent: ComponentType<P>,
  options?: { allowedRoles?: string[] }
) {
  return function WithAuthComponent(props: Omit<P, keyof WithAuthProps>) {
    const { data: session, status } = useSession();
    const router = useRouter();
    
    if (status === 'loading') {
      return <div>Loading...</div>;
    }
    
    if (status === 'unauthenticated') {
      router.push('/login');
      return null;
    }
    
    if (options?.allowedRoles && 
        !options.allowedRoles.includes(session.user.role)) {
      return <div>Access Denied</div>;
    }
    
    return (
      <WrappedComponent 
        {...(props as P)} 
        user={session.user} 
      />
    );
  };
}

// Usage
'use client';

interface DashboardProps extends WithAuthProps {
  initialData: any;
}

function Dashboard({ user, initialData }: DashboardProps) {
  return <div>Welcome {user.email}</div>;
}

export default withAuth(Dashboard, { allowedRoles: ['admin', 'editor'] });
```

### Performance Tracking HOC

Monitor component render performance:

```typescript
// components/with-performance-tracking.tsx
'use client';

import { ComponentType, useEffect, useRef } from 'react';

export function withPerformanceTracking<P extends object>(
  WrappedComponent: ComponentType<P>,
  componentName: string
) {
  return function PerformanceTrackedComponent(props: P) {
    const renderCount = useRef(0);
    const renderStartTime = useRef<number>(0);
    
    useEffect(() => {
      renderStartTime.current = performance.now();
      
      return () => {
        const duration = performance.now() - renderStartTime.current;
        renderCount.current++;
        
        if (duration > 16) { // Longer than one frame
          console.warn(
            `[Performance] ${componentName} render #${renderCount.current} took ${duration.toFixed(2)}ms`
          );
        }
      };
    });
    
    return <WrappedComponent {...props} />;
  };
}

// Usage
export default withPerformanceTracking(HeavyComponent, 'HeavyComponent');
```

## 26.4 Custom Hooks Development

Encapsulate reusable logic in custom hooks following React best practices.

### Async State Hook

Handle async operations with loading/error states:

```typescript
// hooks/use-async.ts
'use client';

import { useState, useCallback, useEffect, useRef } from 'react';

interface UseAsyncOptions<T> {
  immediate?: boolean;
  onSuccess?: (data: T) => void;
  onError?: (error: Error) => void;
}

interface UseAsyncReturn<T, Args extends any[]> {
  execute: (...args: Args) => Promise<T>;
  status: 'idle' | 'pending' | 'success' | 'error';
  data: T | null;
  error: Error | null;
  isLoading: boolean;
}

export function useAsync<T, Args extends any[] = []>(
  asyncFunction: (...args: Args) => Promise<T>,
  options: UseAsyncOptions<T> = {}
): UseAsyncReturn<T, Args> {
  const [status, setStatus] = useState<'idle' | 'pending' | 'success' | 'error'>('idle');
  const [data, setData] = useState<T | null>(null);
  const [error, setError] = useState<Error | null>(null);
  
  const isMounted = useRef(true);
  const lastCallId = useRef(0);
  
  useEffect(() => {
    return () => {
      isMounted.current = false;
    };
  }, []);
  
  const execute = useCallback(
    async (...args: Args): Promise<T> => {
      const callId = ++lastCallId.current;
      
      setStatus('pending');
      setError(null);
      
      try {
        const result = await asyncFunction(...args);
        
        // Ignore if newer call was made or component unmounted
        if (callId !== lastCallId.current || !isMounted.current) {
          return result;
        }
        
        setData(result);
        setStatus('success');
        options.onSuccess?.(result);
        
        return result;
      } catch (err) {
        if (callId !== lastCallId.current || !isMounted.current) {
          throw err;
        }
        
        const error = err instanceof Error ? err : new Error(String(err));
        setError(error);
        setStatus('error');
        options.onError?.(error);
        
        throw error;
      }
    },
    [asyncFunction, options.onSuccess, options.onError]
  );
  
  return {
    execute,
    status,
    data,
    error,
    isLoading: status === 'pending',
  };
}

// Usage
const { execute: fetchUser, isLoading, data } = useAsync(getUser, {
  onSuccess: (user) => toast.success(`Welcome ${user.name}`),
  onError: (err) => toast.error(err.message),
});

// Call manually
<button onClick={() => fetchUser(userId)} disabled={isLoading}>
  Load User
</button>
```

### Intersection Observer Hook

Lazy loading and infinite scroll:

```typescript
// hooks/use-intersection.ts
'use client';

import { useState, useEffect, useRef, RefObject } from 'react';

interface UseIntersectionOptions extends IntersectionObserverInit {
  freezeOnceVisible?: boolean;
}

export function useIntersection<T extends HTMLElement = HTMLDivElement>(
  options: UseIntersectionOptions = {}
): [RefObject<T>, boolean] {
  const { freezeOnceVisible = false, ...observerOptions } = options;
  const ref = useRef<T>(null);
  const [isVisible, setIsVisible] = useState(false);
  
  useEffect(() => {
    const element = ref.current;
    if (!element) return;
    
    // If frozen and already visible, don't observe
    if (freezeOnceVisible && isVisible) return;
    
    const observer = new IntersectionObserver(([entry]) => {
      setIsVisible(entry.isIntersecting);
      
      if (entry.isIntersecting && freezeOnceVisible) {
        observer.unobserve(element);
      }
    }, observerOptions);
    
    observer.observe(element);
    
    return () => observer.disconnect();
  }, [freezeOnceVisible, isVisible, observerOptions.root, observerOptions.rootMargin, observerOptions.threshold]);
  
  return [ref, isVisible];
}

// Usage for lazy loading
function LazyImage({ src, alt }: { src: string; alt: string }) {
  const [ref, isVisible] = useIntersection<HTMLDivElement>({
    threshold: 0.1,
    triggerOnce: true,
  });
  
  return (
    <div ref={ref} className="h-64 bg-gray-200">
      {isVisible && <img src={src} alt={alt} className="w-full h-full object-cover" />}
    </div>
  );
}
```

## 26.5 Headless UI Components

Build accessible, unstyled components that handle behavior while delegating styling.

### Headless Dropdown

Behavior without visual opinions:

```typescript
// components/headless/dropdown.tsx
'use client';

import { useState, useRef, useEffect, createContext, useContext } from 'react';

interface DropdownContextType {
  isOpen: boolean;
  setIsOpen: (open: boolean) => void;
  selectedIndex: number;
  setSelectedIndex: (index: number) => void;
  itemsCount: number;
  registerItem: () => number;
}

const DropdownContext = createContext<DropdownContextType | null>(null);

function useDropdown() {
  const context = useContext(DropdownContext);
  if (!context) throw new Error('Dropdown components must be used within Dropdown');
  return context;
}

// Root
export function Dropdown({ children }: { children: React.ReactNode }) {
  const [isOpen, setIsOpen] = useState(false);
  const [selectedIndex, setSelectedIndex] = useState(-1);
  const itemsCountRef = useRef(0);
  
  const registerItem = () => itemsCountRef.current++;
  
  return (
    <DropdownContext.Provider
      value={{
        isOpen,
        setIsOpen,
        selectedIndex,
        setSelectedIndex,
        itemsCount: itemsCountRef.current,
        registerItem,
      }}
    >
      <div className="relative">{children}</div>
    </DropdownContext.Provider>
  );
}

// Trigger
interface TriggerProps {
  children: (props: { isOpen: boolean }) => React.ReactNode;
}

export function Trigger({ children }: TriggerProps) {
  const { isOpen, setIsOpen } = useDropdown();
  
  return (
    <button
      type="button"
      onClick={() => setIsOpen(!isOpen)}
      aria-expanded={isOpen}
    >
      {children({ isOpen })}
    </button>
  );
}

// Menu
interface MenuProps {
  children: React.ReactNode;
  className?: string;
}

export function Menu({ children, className }: MenuProps) {
  const { isOpen } = useDropdown();
  
  if (!isOpen) return null;
  
  return (
    <div className={className} role="menu">
      {children}
    </div>
  );
}

// Item
interface ItemProps {
  children: React.ReactNode;
  onSelect: () => void;
  disabled?: boolean;
  className?: string;
}

export function Item({ children, onSelect, disabled, className }: ItemProps) {
  const { registerItem, setIsOpen } = useDropdown();
  const index = useRef(registerItem()).current;
  
  const handleClick = () => {
    if (disabled) return;
    onSelect();
    setIsOpen(false);
  };
  
  return (
    <div
      role="menuitem"
      className={className}
      onClick={handleClick}
      aria-disabled={disabled}
    >
      {children}
    </div>
  );
}
```

### Styled Implementation

Consume headless components with your design system:

```typescript
// components/ui/dropdown.tsx
import * as Headless from '@/components/headless/dropdown';
import { ChevronDown } from 'lucide-react';

export function StyledDropdown() {
  return (
    <Headless.Dropdown>
      <Headless.Trigger>
        {({ isOpen }) => (
          <div className="flex items-center gap-2 px-4 py-2 bg-white border rounded-md shadow-sm hover:bg-gray-50">
            Options
            <ChevronDown className={`w-4 h-4 transition-transform ${isOpen ? 'rotate-180' : ''}`} />
          </div>
        )}
      </Headless.Trigger>
      
      <Headless.Menu className="absolute z-50 w-full mt-2 bg-white border rounded-md shadow-lg">
        <Headless.Item 
          onSelect={() => console.log('edit')}
          className="px-4 py-2 hover:bg-gray-100 cursor-pointer first:rounded-t-md"
        >
          Edit
        </Headless.Item>
        <Headless.Item 
          onSelect={() => console.log('delete')}
          disabled
          className="px-4 py-2 hover:bg-gray-100 cursor-pointer opacity-50 last:rounded-b-md"
        >
          Delete
        </Headless.Item>
      </Headless.Menu>
    </Headless.Dropdown>
  );
}
```

## 26.6 Server Component Patterns

Leverage Server Components for data fetching and composition.

### Server Component with Client Islands

Pattern for mixing server and client rendering:

```typescript
// app/dashboard/page.tsx (Server Component)
import { getDashboardData } from '@/lib/data';
import { StatsWidget } from './stats-widget'; // Server
import { RealtimeChart } from './realtime-chart'; // Client
import { InteractiveFilters } from './interactive-filters'; // Client

export default async function DashboardPage() {
  // Fetch on server
  const data = await getDashboardData();
  
  return (
    <div className="grid grid-cols-3 gap-6">
      {/* Static content - rendered on server */}
      <StatsWidget stats={data.stats} />
      <StatsWidget stats={data.revenue} />
      
      {/* Client interactivity island */}
      <RealtimeChart initialData={data.chartData} />
      
      {/* Client component with server data */}
      <InteractiveFilters 
        categories={data.categories}  // Serialized from server
        initialSelection={data.defaultCategory}
      />
    </div>
  );
}

// components/dashboard/realtime-chart.tsx
'use client';

import { useEffect, useState } from 'react';

export function RealtimeChart({ initialData }: { initialData: any[] }) {
  const [data, setData] = useState(initialData);
  
  // Client-side polling for real-time updates
  useEffect(() => {
    const interval = setInterval(async () => {
      const fresh = await fetch('/api/chart-data').then(r => r.json());
      setData(fresh);
    }, 5000);
    
    return () => clearInterval(interval);
  }, []);
  
  return <Chart data={data} />;
}
```

### Async Server Components with Suspense

Parallel data fetching with granular loading states:

```typescript
// app/products/page.tsx
import { Suspense } from 'react';
import { ProductListSkeleton } from './product-list-skeleton';
import { FilterSidebarSkeleton } from './filter-skeleton';

// These components fetch their own data
async function ProductList({ category }: { category: string }) {
  const products = await fetch(`https://api.example.com/products?category=${category}`, {
    cache: 'no-store',
  }).then(r => r.json());
  
  return (
    <div className="grid grid-cols-3 gap-4">
      {products.map((p: any) => (
        <ProductCard key={p.id} product={p} />
      ))}
    </div>
  );
}

async function FilterSidebar() {
  const categories = await fetch('https://api.example.com/categories', {
    next: { revalidate: 3600 },
  }).then(r => r.json());
  
  return <FilterOptions categories={categories} />;
}

export default function ProductsPage({ 
  searchParams 
}: { 
  searchParams: { category?: string } 
}) {
  return (
    <div className="flex gap-8">
      <aside className="w-64">
        <Suspense fallback={<FilterSidebarSkeleton />}>
          <FilterSidebar />
        </Suspense>
      </aside>
      
      <main className="flex-1">
        <Suspense fallback={<ProductListSkeleton count={6} />}>
          <ProductList category={searchParams.category || 'all'} />
        </Suspense>
      </main>
    </div>
  );
}
```

## 26.7 Performance Component Patterns

Optimize component rendering for smooth user experiences.

### Virtualization for Large Lists

Render only visible items:

```typescript
// components/virtual-list.tsx
'use client';

import { useRef, useEffect, useState } from 'react';

interface VirtualListProps<T> {
  items: T[];
  itemHeight: number;
  renderItem: (item: T, index: number) => React.ReactNode;
  overscan?: number;
}

export function VirtualList<T>({ 
  items, 
  itemHeight, 
  renderItem,
  overscan = 5 
}: VirtualListProps<T>) {
  const containerRef = useRef<HTMLDivElement>(null);
  const [scrollTop, setScrollTop] = useState(0);
  const [containerHeight, setContainerHeight] = useState(0);
  
  useEffect(() => {
    if (!containerRef.current) return;
    
    const resizeObserver = new ResizeObserver(([entry]) => {
      setContainerHeight(entry.contentRect.height);
    });
    
    resizeObserver.observe(containerRef.current);
    return () => resizeObserver.disconnect();
  }, []);
  
  const totalHeight = items.length * itemHeight;
  const startIndex = Math.max(0, Math.floor(scrollTop / itemHeight) - overscan);
  const visibleCount = Math.ceil(containerHeight / itemHeight) + overscan * 2;
  const endIndex = Math.min(items.length, startIndex + visibleCount);
  
  const visibleItems = items.slice(startIndex, endIndex);
  const offsetY = startIndex * itemHeight;
  
  return (
    <div
      ref={containerRef}
      style={{ height: '100%', overflow: 'auto' }}
      onScroll={(e) => setScrollTop(e.currentTarget.scrollTop)}
    >
      <div style={{ height: totalHeight, position: 'relative' }}>
        <div style={{ transform: `translateY(${offsetY}px)` }}>
          {visibleItems.map((item, index) => (
            <div key={startIndex + index} style={{ height: itemHeight }}>
              {renderItem(item, startIndex + index)}
            </div>
          ))}
        </div>
      </div>
    </div>
  );
}

// Usage with 10,000 items
<VirtualList
  items={largeDataset}
  itemHeight={80}
  renderItem={(item, index) => (
    <div className="p-4 border-b">
      <h3>{item.title}</h3>
      <p>Item #{index}</p>
    </div>
  )}
/>
```

### Batching and Debouncing

Optimize high-frequency updates:

```typescript
// hooks/use-debounce-callback.ts
'use client';

import { useRef, useCallback } from 'react';

export function useDebouncedCallback<T extends (...args: any[]) => void>(
  callback: T,
  delay: number,
  options: { leading?: boolean; trailing?: boolean } = {}
) {
  const timeoutRef = useRef<NodeJS.Timeout>();
  const lastCallTime = useRef<number>(0);
  
  return useCallback(
    (...args: Parameters<T>) => {
      const now = Date.now();
      const isLeading = options.leading && now - lastCallTime.current > delay;
      
      lastCallTime.current = now;
      
      if (timeoutRef.current) {
        clearTimeout(timeoutRef.current);
      }
      
      if (isLeading) {
        callback(...args);
      }
      
      timeoutRef.current = setTimeout(() => {
        if (options.trailing !== false && !isLeading) {
          callback(...args);
        }
        timeoutRef.current = undefined;
      }, delay);
    },
    [callback, delay, options.leading, options.trailing]
  );
}

// Usage for search input
const debouncedSearch = useDebouncedCallback((query: string) => {
  router.push(`?search=${encodeURIComponent(query)}`);
}, 300, { trailing: true });
```

## Key Takeaways from Chapter 26

1. **Compound Components**: Build flexible component APIs by creating related components that share implicit state through React Context. Export components as a namespace object (e.g., `Select.Root`, `Select.Item`) to indicate their relationship while allowing flexible composition and styling.

2. **Render Props**: Use the render props pattern when you need to share code between components without knowing the exact UI structure ahead of time. This pattern excels for data fetching, mouse tracking, and layout components where the consumer determines the rendering strategy.

3. **Higher-Order Components**: Wrap components with HOCs to inject cross-cutting concerns like authentication, performance tracking, or error boundaries. Preserve the wrapped component's display name for debugging, and properly type the HOC to maintain TypeScript intellisense for both required and injected props.

4. **Custom Hooks**: Encapsulate reusable stateful logic in custom hooks following the `use[Name]` convention. Handle race conditions with request IDs or AbortController, track mount status to prevent state updates on unmounted components, and provide ergonomic APIs with loading states and error handling.

5. **Headless UI**: Separate behavior from presentation by creating headless components that manage state, accessibility, and interactions while delegating all styling to the consumer. This enables design system flexibility while ensuring consistent accessibility patterns and keyboard navigation.

6. **Server Component Patterns**: Architect pages as Server Components that fetch initial data, passing serialized data to Client Component "islands" for interactivity. Use Suspense boundaries to parallelize data fetching and show granular loading states, keeping heavy JavaScript out of the initial bundle.

7. **Performance Patterns**: Implement virtualization for lists with more than 50 items to render only visible content. Use `useDeferredValue` and `useTransition` for non-urgent updates, and debounce high-frequency events (scroll, resize, input) to prevent excessive re-renders and maintain 60fps interactions.

## Coming Up Next

**Chapter 27: Real-Time Features**

Now that you've mastered component architecture patterns, it's time to add dynamic, live functionality to your applications. In Chapter 27, we'll explore WebSockets integration, Server-Sent Events (SSE), real-time data synchronization, live updates implementation, push notifications, collaborative features, and real-time best practices. You'll learn how to build applications that respond instantly to data changes across connected clients.